In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab/transformer/yolov5

/content/drive/MyDrive/Colab/transformer/yolov5


In [ ]:
!pip install -r requirements.txt

In [3]:
cd /content/drive/MyDrive/Colab/transformer/transfg

/content/drive/MyDrive/Colab/transformer/transfg


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 776.7 MB 4.7 kB/s 
     |████████████████████████████████| 12.7 MB 33.7 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=ead4818660a84ef3302d0e8ab33c4b587545edaeaffe097bc39d807cddde99a8
  Stored in directory: /root/.cache/pip/wheels/b7/da/64/33c926a1b10ff19791081b705879561b715a8341a856a3bbd2
Successfully built ml-collections
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab/transformer/yolov5')
sys.path.insert(0,'/content/drive/MyDrive/Colab/transformer/transfg/models_trans')
sys.path

['/content/drive/MyDrive/Colab/transformer/transfg/models_trans',
 '/content/drive/MyDrive/Colab/transformer/yolov5',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [3]:
import torch
print(torch.__version__)

1.7.0


In [4]:
cd /content/drive/MyDrive/Colab/transformer/yolov5

/content/drive/MyDrive/Colab/transformer/yolov5


In [5]:
detector = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-3-31 torch 1.7.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [6]:
cd /content/drive/MyDrive/Colab/transformer/transfg

/content/drive/MyDrive/Colab/transformer/transfg


In [7]:
from __future__ import absolute_import, division, print_function
import models_trans.configs as configs
import logging
import argparse
import os
import random
import numpy as np
import time
from PIL import Image
from datetime import timedelta
import torch
import pandas as pd
from torch.utils.data import Dataset
from torchvision import transforms
from models_trans.modeling import VisionTransformer, CONFIGS
from models_trans.dog_class import classes

CONFIGS = {
    'ViT-B_16': configs.get_b16_config(),
    'ViT-B_32': configs.get_b32_config(),
    'ViT-L_16': configs.get_l16_config(),
    'ViT-L_32': configs.get_l32_config(),
    'ViT-H_14': configs.get_h14_config(),
    'testing': configs.get_testing(),
}


class FineGrainDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None, test=False):
        self.root_dir = root_dir  # 圖片本人路徑
        self.annotations = pd.read_csv(annotation_file)  # 上一步做的CSV
        self.transform = transform  # 定義要做的transform, 含有resize把圖片先resize成依樣
        self.test = test

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]  # 取出image id => ex: 0003.jpg
        if self.test == False:
            img = Image.open(os.path.join(self.root_dir, img_id)).convert(
                "RGB")  # 取出image id 對應的圖片本人, 並且轉RGB(等等用transform來轉tensor)
            temp = self.annotations.iloc[index, 1]
            y_label = torch.tensor(self.annotations.iloc[index, 1]).long()
            img = self.transform(img)
            return (img, y_label - 1)
        else:
            # print("fetch testing image id: ", img_id)
            img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
            img = self.transform(img)
            return (img, img_id)

#training model parameter setting
config = CONFIGS["ViT-B_16"]
config.slide_step = 12
config.split = 'overlap'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#inference data transform
test_transform = transforms.Compose([transforms.Resize((600, 600), Image.BILINEAR),transforms.CenterCrop((348, 348)),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

#Pretrained TransFG mdoel prepared
pretrained_model_path = "./output/transfg.bin"
model = VisionTransformer(config, 348, zero_head=True, num_classes=120, smoothing_value=0.0)
if pretrained_model_path is not None:
  pretrained_model = torch.load(pretrained_model_path)['model']
  model.load_state_dict(pretrained_model)
model.to(device)
model.eval()
print('model prepared')

model prepared


In [8]:
!pip install anvil-uplink
import anvil.server
anvil.server.connect("server_VFL2ORJJKT22YWR7YJEL2AVN-4TTOM7KQI4VYB57T")

     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 51 kB 232 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=c25bb1f207390835d94e0f807090fdebe2fb9f8d0b94233035e7ba62b665d2d8
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [32]:
import anvil.media
import anvil.mpl_util
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
from PIL import Image

@anvil.server.callable
def dog_classifier(file):
  with anvil.media.TempFile(file) as filename: # getting upload images from the anvil server. 
    img = Image.open(filename).convert('RGB') 
    results = detector(img) # using yolov5 to get the detection result
    fig, ax = plt.subplots()
    ax.imshow(img)
    plt.axis('off')

  if len(results.xyxy[0]) == 0: # nothing detected
    plt.savefig('pics/{}{}.png'.format('blank',random.randint(0,1000))) # save images 
    return 'Nothing detected!', 0, anvil.mpl_util.plot_image()

  else:
    dog = 0
    for i in range(len(results.xyxy[0])):
      if results.pandas().xyxy[0].iloc[i]['name']=='dog': # dog detected in the image
        dog = 1 
        if results.pandas().xyxy[0].iloc[i]['confidence'] > 0.1: # setting confidence threshold
          # get the bounding box
          xmin,ymin,xmax,ymax = results.pandas().xyxy[0].iloc[i]['xmin'],results.pandas().xyxy[0].iloc[i]['ymin'],results.pandas().xyxy[0].iloc[i]['xmax'],results.pandas().xyxy[0].iloc[i]['ymax'] 
          # crop the image
          img_crop = img.crop((results.pandas().xyxy[0].iloc[i]['xmin'],results.pandas().xyxy[0].iloc[i]['ymin'],results.pandas().xyxy[0].iloc[i]['xmax'],results.pandas().xyxy[0].iloc[i]['ymax']))
          # plot rectangle
          rect = patches.Rectangle((xmin,ymin),xmax-xmin,ymax-ymin, linewidth=3, edgecolor='r', facecolor='none')
          ax.add_patch(rect)
          # image transform
          x = test_transform(img_crop)
          # get prediction
          test_pred = model(x.unsqueeze(0).cuda())
          # get label
          test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
          probs = torch.nn.Softmax(dim=-1)(test_pred)
          # save image
          plt.savefig('pics/{}{}.png'.format(classes[test_label[-1]],random.randint(0,1000)))
          # return dog breed that is predicted by the model
          return classes[test_label[-1]],round(float(probs[0][int(test_label)]),3),anvil.mpl_util.plot_image()
    if dog == 0: # no dog detected in the image
      return_string = 'No dog detected, instead detect a {}!'.format(results.pandas().xyxy[0].iloc[0]['name']) # return the object with highest confidence score 
      plt.savefig('pics/{}{}.png'.format(results.pandas().xyxy[0].iloc[0]['name'],random.randint(0,1000)))
      xmin,ymin,xmax,ymax = results.pandas().xyxy[0].iloc[0]['xmin'],results.pandas().xyxy[0].iloc[0]['ymin'],results.pandas().xyxy[0].iloc[0]['xmax'],results.pandas().xyxy[0].iloc[0]['ymax']
      rect = patches.Rectangle((xmin,ymin),xmax-xmin,ymax-ymin, linewidth=3, edgecolor='g', facecolor='none')
      ax.add_patch(rect)
      return return_string , results.pandas().xyxy[0].iloc[0]['confidence'] ,anvil.mpl_util.plot_image()

In [10]:
import glob
test_google = glob.glob('/content/drive/MyDrive/Colab/transformer/test/*')

In [ ]:
for i in range(10):
  test1 = test_google[i]+'/*'
  test = glob.glob(test1)
  model.to(device)
  model.eval()
  for i,f in enumerate(range(len(test))):
    try:
      img = Image.open(test[f]).convert('RGB')
      x = test_transform(img)
      test_pred = model(x.unsqueeze(0).cuda())
      test_label = np.argsort(test_pred.cpu().data.numpy().flatten())[-1]
      print(i,classes[test_label])
    except:
      pass